<a href="https://colab.research.google.com/github/Yatharths11/Books/blob/master/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LangChain Frameworks

##Q0: Prepation code.

Installing necessary packages.

In [55]:
!pip install langchain
!pip install pypdf
!pip install openai
!pip install tiktoken
!pip install faiss-cpu
!pip install nltk
!pip install pandas
!pip install --upgrade PyPDF2

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Retrieve OpenAI API key.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [32]:
config_ini_location = '/content/config.ini' # Change this to point to the location of your config.ini file.

import configparser

config = configparser.ConfigParser()
config.read(config_ini_location)
openai_api_key = config['OpenAI']['API']

For this assignment you will use ``model_name="gpt-3.5-turbo-0613"`` only. **You are NOT allowed to use any other model. You will lose 1 point per question if you violate this requirement.**

In [ ]:
model_name="gpt-3.5-turbo-0613" # Do Not change this!

**For debugging purposes for all the questions below, remember that using `verbose`  and `langchain.debug` to print the actual requests and responses is quite useful.**

## Q1:  Question Answering System Using the School's Syllabus Database (4.5 points)

 At your school, the department has embarked on a project to utilize language modeling for the development of a question-answering agent. This initiative aims to streamline the access to information for faculty and staff, particularly regarding the extensive array of courses offered at our institution. The data pertaining to these courses is currently dispersed across numerous documents within [the department's syllabus corpus](https://drive.google.com/drive/folders/1dH-t_Ujih4lMMzUOaNOHngvOYLK_gWOp?usp=sharing).

Download the corpus to your Google Drive and update the path below.

Note: The used syllabus corpus is a subset of [Cal Poly's Syllabus Corpus dataset](https://www.kaggle.com/datasets/mfekadu/syllabus-corpus).

In [ ]:
syllabus_corpus_path = "/content/drive/MyDrive/Colab Notebooks/IS883/IS883_HW4_syllabus_corpus/"

First, you will use a [PyPDFDirectoryLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain.document_loaders.pdf.PyPDFDirectoryLoader.html) to create a loader that can load all the PDFs in the directory so they could be used by LangChain.

In [46]:
import pandas as pd
import numpy as np
import os
import requests
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.docstore.document import Document

In [65]:
from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader('/content/drive/MyDrive/archive (9)/downloads')
docs = loader.load()


PdfStreamError: ignored

Given the extensive data contained within these documents, it's impractical to include them in their entirety in our queries. Including all data at once could exceed the context window's capacity and may result in significant processing costs. To address this challenge, you will employ a methodical approach to manage the data effectively.

* Create a [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter): You will use a `RecursiveCharacterTextSplitter` to divide the documents into more manageable segments. This splitter will break down the documents into chunks.

* Configurations: **(0.25 point)**
  * Chunk Size Configuration: Set the `chunk_size` to 500 characters. This size ensures that the chunks are large enough to contain meaningful content but small enough to be processed efficiently.

  * Creating Overlapping Chunks: Set `chunk_overlap` to 50 characters. This overlap will help prevent the loss of context that might occur at the boundaries of each chunk. It ensures that no critical information is missed or misunderstood due to the chunking process.

In [58]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 50,
)

Now, using the afortmentioned loader and splitter, perform the splitting.

In [61]:
loader.load_and_split(text_splitter)

PdfStreamError: ignored

The next crucial step involves the creation of a data store, essentially a database, that will house the chunks of data you've created. The effectiveness of our question-answering system hinges on its ability to swiftly locate the relevant chunk containing the answer to any given query. To achieve this efficiency, we will employ a sophisticated indexing strategy, rather than relying on a basic brute-force search method.

* Build the Data Store with [Facebook AI Similarity Search (FAISS)](https://python.langchain.com/docs/integrations/vectorstores/faiss): Set up your data store using a [FAISS Vector store](https://python.langchain.com/docs/integrations/vectorstores/faiss). FAISS is a library developed by Facebook AI that allows for efficient similarity search and clustering of dense vectors.

* Embedding Calculation with `OpenAIEmbeddings`: For each chunk of data in your store, calculate an embedding using `OpenAIEmbeddings`. These embeddings are essentially numerical representations of your text data, which can then be compared to the embeddings of incoming queries.

* Indexing for Efficient Search: By creating embeddings for each chunk and indexing them in the FAISS Vector store, you will enable the system to quickly find the most relevant chunk in response to a query. This process involves comparing the embedding of the query with the embeddings of the chunks to identify the best match.

The combination of `FAISS` and `OpenAIEmbeddings` will significantly enhance the efficiency and accuracy of the question-answering system, allowing for rapid retrieval of information from the extensive syllabus corpus.

With the data store and indexing system in place, you are now equipped to tackle the core functionality of our question-answering system: responding to queries based on the indexed database.

* Utilize the [*`similarity_search`*](https://python.langchain.com/docs/integrations/vectorstores/faiss) function to identify the chunk that is most relevant or most similar to the posed question. This function will compare the embedding of the query with those of the indexed chunks to find the best match. **(0.25 point)**

* Display Source Information: Once you have identified the most relevant answer, output additional details indicating where this chunk is located. Specifically, provide information about *the page number and the document from which this chunk was extracted*. **(0.5 point)**

To gain a deeper understanding of how similarity search operates, refer to the provided articles and references. These resources will offer a more detailed conceptual insight into the workings of similarity search algorithms and their applications in systems like ours.

[Resource 1.](https://www.pinecone.io/learn/what-is-similarity-search/)

[Resource 2.](https://python.langchain.com/docs/modules/data_connection/vectorstores/)

In [ ]:
question = "Who is the instructor of Linear Algebra III?"

Next, you will delve deeper into the results to evaluate the system.

* Display the Top 5 Matches: Print the top five most relevant chunks in response to your query, *along with their respective similarity scores*. These scores quantify how closely each chunk matches your query, offering a clear metric of relevance. **(0.5 point)**



* Examine why certain chunks received higher or lower similarity scores. Analyze the content of each chunk in relation to your query to understand the basis of these scores. **(0.25 point)**

  * Discuss whether the model is effectively discerning relevant information or if it appears to be misled by certain elements. Provide suggestions for improvements.

[Resource.](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.faiss.FAISS.html)

Finally, we are going to use OpenAI API to get the answer to the question based on the relevant chunk. To do that, we will LangChain's *load_qa_chain*. This [article](https://cloudatlas.me/query-your-pdfs-with-openai-langchain-and-faiss-7e8221791c62) should give you an example of how to use it.

The final step involves leveraging OpenAI API to obtain answers to your queries based on the top `k` most relevant chunk identified in the previous step **(0.25 point)**. For this, you will use LangChain's [`load_qa_chain`](https://cloudatlas.me/query-your-pdfs-with-openai-langchain-and-faiss-7e8221791c62) functionality.

* Utilize `load_qa_chain` to integrate OpenAI API into your question-answering system. This tool will enable you to send the selected chunk as a context to the API and retrieve a "precise" answer to your query.

* Track the requests sent and the responses received from the OpenAI API. This will give you visibility into the interaction between your system and the API. **(0.25 point)**

* Analyze the requests and responses in detail. Discuss how the API processes the chunk and formulates an answer **(0.5 point)**. Evaluate the overall performance of the system in leveraging OpenAI API for answering queries. Consider the relevance and precision of the answers, and how well the system integrates the information from the chunks to generate responses. **(0.5 point)**



In [ ]:
temperature =

In [ ]:
from langchain.chat_models import ChatOpenAI

# Create a reference to the language model
llm = ChatOpenAI(openai_api_key=openai_api_key, temperature=temperature, model_name=model_name)

**Answer:**



It's important to analyze and compare the system's performance across various questions.



* Compare with First Question: Reflect on the system's response to the following question and compare it with the response to the first question above. Note any differences in accuracy, relevance, or clarity of the answers. **(0.5 point)**

* Analyze the causes behind these observations. Consider factors such as the nature of the question, the relevance of the chosen chunk, and how the AI model interprets different types of queries. **(0.25 point)**

* Propose Changes: Based on your observations, propose potential changes or adjustments that could improve the system's ability to retrieve more accurate or relevant answers **(0.25 point)**. Evaluate Trade-offs: Discuss the trade-offs associated with the changes you propose. **(0.25 point)**


In [ ]:
question2 = "What additional cost does Lean Six Sigma Black Belt Training require?"

**Answer:**


##Q2: AI-Powered "I went to the beach" Showdown (4.5 points)

In this part, you will explore the realm of AI-driven creativity by setting up an engaging interaction between two chatbots. The aim is to test the extent of the AI's "perpetual" creativity in a fun and interactive setup. You will create a scenario where two chatbots complete each other's sentences in a themed showdown revolving around the phrase "I went to the beach."

In [ ]:
first_sentence = "I went to the beach with my son and daughter."

In [ ]:
temperature = 0.3

First, Create the `ChatOpenAI` object. Be sure to use the `model_name` above.

In [ ]:
from langchain.chat_models.openai import ChatOpenAI

llm = ChatOpenAI(temperature=temperature,openai_api_key=openai_api_key, model_name=model_name)

The next step is to construct the first AI participant, referred to as **AI_1**.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.schema import HumanMessage, SystemMessage, AIMessage

# Create the prompt
template = """We are going to take turns building a story. The story should not be repetitive. It should keep adding new events.
Each turn, I will first give a sentance of no more than 20 words, and then you will give a sentence of a similar length. We repeat the cycle indefinitely.

{chat_history}
Human: {human_input}
AI:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)

# create the memory of the conversation
memory = ConversationBufferMemory(memory_key="chat_history")

# create the chain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

# get the first AI reponse
response1 = llm_chain.predict(human_input=first_sentence)
print("AI_1:", response1)

Let's look at the conversation's memory from **AI_1**'s perspective. This should print the entire conversation **(0.25 point)**. It should look like this:

```
I went to the beach with my son and daughter.
We built sandcastles and played in the waves, laughing and splashing each other <<or something similar...>>
```

After successfully setting up **AI_1**, the next step is to develop the second participant in the creative exchange, **AI_2**. This AI will have roles that are opposite to those of **AI_1** (i.e., "Human" here is "AI" there and vice versa.).

* Setting Up `ConversationBufferMemory` (memory2): Establish a new `ConversationBufferMemory`, named `memory2`, for **AI_2**. This memory will be responsible for storing the conversation as perceived and processed by **AI_2**.
* Initializing Memory with First Sentence: Incorporate an `AIMessage`: Ensure that `memory2` already contains an `AIMessage` corresponding to the first sentence of the conversation corresponding to **AI_1**'s first response. This is crucial for setting the context from **AI_2**’s perspective. Utilize the [`add_ai_message`](https://python.langchain.com/docs/modules/memory/) function to add this initial message to `memory2`.

Consult the [provided reference](https://python.langchain.com/docs/modules/memory/conversational_customization) for a thorough understanding of message manipulation and conversation customization.

After setting up the foundational elements for **AI_2**, your next task is to build its conversational chain, using `LLMChain` and use it to generate the next sentence in the "I went to the beach" showdown. **(0.5 point)**

At the end of your setup, `memory2.chat_memory` should have the following messages:


Conclude the setup for **AI_2** by verifying that the content of `memory2.chat_memory` correctly reflects the conversation **(0.25 point)**. It should contain the following sequence of messages:



```
I went to the beach with my son and daughter.
<<AI_1's response>>.
<<AI_2's reponse>>.
```

Now, let's create a loop where **AI_1** and **AI_2** engage in a series of sentence completions. You'll also be tracking the total cost of the entire conversation using the `get_openai_callback` function.



* Create a loop that runs for 10 iterations. In each iteration, **AI_1** and **AI_2** will take turns completing each other’s sentences, contributing to the ongoing conversation. **(0.5 point)**

* Within the loop, after each AI has generated its response, print the responses along with the iteration number in the following format:
```
iteration X
AI_1: <<AI_1's response>>
AI_2: <<AI_2's response>>
```
Replace X with the current iteration number, and ensure the responses are accurately captured. **(0.25 point)**

* Use the [`get_openai_callback`](https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking) to track the **total** cost for the entire conversation, as opposed to per-iteration costs. **(0.25 point)**



In [ ]:
iterations=10

For optimizing the token usage and cost while preserving the essence of the conversation, you'll need to adapt your code to incorporate [`ConversationSummaryBufferMemory`](https://python.langchain.com/docs/modules/memory/types/summary_buffer). This memory type will summarize the conversation when a predefined `max_token_limit=200` is reached, thus reducing the token count sent to the language model in each iteration.

* While keeping the existing language model object (`llm`) intact, recreate the chains for **AI_1** and **AI_2** to use the new `ConversationSummaryBufferMemory`. **(0.25 point)**
* Repeat the same experiment above (i.e., conducting a 10-round conversation) using the new chains. **(0.5 point)**

For a better understanding of the different types of memory and their functionalities, refer to [the following article](https://www.pinecone.io/learn/series/langchain/langchain-conversational-memory/).



In [ ]:
max_token_limit=

In [ ]:
# Define AI_1 and produce its first sentence.


# Define AI_2 and produce its first sentence.



# Get the conversation going.


Answer the following questions:

* Having run the conversational AI using both standard memory and `ConversationSummaryBufferMemory`, compare the total cost incurred with each type of memory. Which memory type results in low cost? Discuss the causes behind your findings. **(0.5 point)**

* [Measure the time](https://www.programiz.com/python-programming/examples/elapsed-time) taken to execute the conversation loop with each type of memory. Determine which memory type takes longer to execute and provide reasons for this. **(0.25 point)**

* Observe the final conversations for both types of memory. Provide evidence from the code's output showing how the summary is generated, and how the conversation is condensed once the `max_token_limit` is reached. **(0.25 point)**

* Describe the process of summary generation. Assess the effectiveness of the summary in representing the original conversation. **(0.5 point)**

* What prompt engineering framework was used in `ConversationSummaryBufferMemory`? Provide evidence. **(0.25 point)**



**Answer:**



##Q3 (BONUS): Riddle Me This... **(4.5 points)**

In this question, you will get to have some fun with math riddles and explore the impact of different prompt engineering frameworks on solving them using AI.

In [ ]:
riddle  = "A man left 100 dollars to be divided between his two sons Alfred and Benjamin. If one third of Alfred’s legacy was taken from one-fourth of Benjamin’s, the remainder would be 11 dollars. How much is Alfred's legacy?"

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools

**Solution 0: Zero-shot learning**.

* Create an [OpenAI](https://api.python.langchain.com/en/latest/llms/langchain.llms.openai.OpenAI.html) language model object.
* Decide on a temperature value for the model. Explain your choice.
* Use the language model to get a response to the riddle. No prompt engineering at this point.
* Note AI's answer to the riddle. Is it correct or not? Discuss your observations on how well the model responded to the riddle. **(0.25 point)**



In [ ]:
temperature=

In [ ]:
## Create the llm
from langchain.llms import OpenAI
llm = OpenAI(temperature=, openai_api_key=openai_api_key)

**Answer**:

**Solution 1: Zero-shot learning with a calculator**.


Let's try enhancing the zero-shot learning approach by integrating a calculator tool with the language model. This setup aims to improve the accuracy and effectiveness of solving riddles, especially those involving mathematical elements.

* Using [`load_tools`](https://python.langchain.com/docs/modules/agents/tools/), load the `llm-math` tool, which will serve as a calculator. This tool can be integrated with the language model to handle any mathematical computations required in the riddle-solving process.

* Using [`initialize_agent`](https://python.langchain.com/docs/modules/agents/agent_types/openai_functions_agent#using-openaifunctionsagent), create a **zero-shot agent** with the `initialize_agent` function.




Invoke the agent with the riddle.


* Show evidence that the calculator tool was utilized to obtain the response by examining the intermediate steps.  **(0.5 point)**
* Evaluate whether the answer provided by the agent is correct. **(0.25 point)**
* Reflect on whether you expected the calculator to aid in solving this riddle. Discuss if the calculator tool has actually contributed to finding the solution. Include in your dicussion an explanation of how and why (or why not). **(0.5 point)**

* What prompt engineering framework was used here? Provide evidence. **(0.25 point)**


**Answer:**



**Solution 3: Few-shot Learning**.

Now, you'll provide the model with examples to learn from. This approach aims to improve AI's ability to solve riddles by presenting it with similar problems and their solutions.

* Create three examples of riddles that are similar to the one you are given. For each example riddle, only provide a clear and correct *final* solution. This will serve as a learning reference for the AI. **(0.25 point)**
* Create a `PromptTemplate` that formats the example riddles and their solutions appropriately. Utilize the prompt to create a `FewShotPromptTemplate`. This template will be used to present the example riddles and their solutions to the AI in a way that facilitates learning. Refer to [this reference](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/few_shot_examples) for guidance.
* Introduce slight variations in the answers to reduce *social* biases. Consider adding other variations that could enhance learning. Discuss your implementation. **(0.25 point)**

**Answer**:

Now, call the `llm` object with the template after properly substituting the riddle into it.


* Explain how you obtained the used examples. **(0.25 point)**
* Show evidence that the template was utilized to obtain the response by examining the intermediate steps.  **(0.25 point)**
* Evaluate whether the answer provided by the framework is correct. **(0.25 point)**
* Reflect on whether you expected the framework to aid in solving this riddle. Discuss if it has actually contributed to finding the solution. Include in your dicussion an explanation of how and why (or why not). **(0.5 point)**

**Answer:**



**Solution 3: Chain of Thought (CoT)**.


Finally, you'll guide the model through a step-by-step process, breaking down the solution into clear, logical steps. This CoT approach helps the AI model understand the reasoning process needed to arrive at the correct answer.


* Create a detailed example of a similar riddle and logically break it down into steps. The steps involve translating the problem into an alogrithm with equation solving. Clearly outline each step. **(0.25 point)**
* Ensure that each step in the solution is explained clearly and simply. Avoid complex jargon or overly technical language. Make sure the steps follow a logical sequence that is easy to understand and replicate.
* Use the same `PromptTemplate` object from the previous solution to maintain consistency in formatting.
* Create a new `FewShotPromptTemplate` that incorporates the detailed example you've created. This template will guide the AI model in applying a similar chain of thought to solve other riddles.

Now, call the `llm` object with the template after properly substituting the riddle into it.


* Evaluate whether the answer provided by the framework is correct. **(0.25 point)**
* Reflect on whether you expected the framework to aid in solving this riddle. Discuss if it has actually contributed to finding the solution. Include in your dicussion an explanation of how and why (or why not). **(0.5 point)**

**Answer:**

